In [4]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
from webdriver_manager.chrome import ChromeDriverManager
import time
from indicnlp import common
import pandas as pd
import re
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.by import By
from langdetect import detect

import numpy as np

import multiprocessing
import os
import torch

In [5]:
pd.set_option('display.max_rows', 500)

In [6]:
num_cpu_input=2
path_dir=os.getcwd()+'/indicTrans'           #'path/to/module/indicTrans/'
save_dir=os.getcwd()+'/data'                 #'path/to/dir/for/saving/CSVs'
batch_size=10
device='cuda:0'

<h1><center>Task 1 & Task 2</center></h1>

In this section, following actions are performed - 
1) Web scraping for all 21 Indic languages and English
2) Some processing to get rid of discrepancy of number of rows in the data across languages
3) Language detection using langdetect

Note: user-defined functions present in **web_scraper_func.py** (file included in the github repo)

In [ ]:
import multiprocessing

In [ ]:
import web_scraper_func

In [ ]:
languages = ['English','हिंदी','ગુજરાતી','મરાઠી','ಕನ್ನಡ','മലയാളം','தமிழ்','తెలుగు','বাংলা','অসমীয়া','ਪੰਜਾਬੀ','ଓଡ଼ିଆ','नेपाली','डोगरी','कोंकणी','सिंधी','बोडो','मैथिली','মণিপুরী','संथाली','اردو','کٲشُر']

num_cpu_input=1

if os.cpu_count()>num_cpu_input:
    num_cpu=num_cpu_input
else:
    num_cpu=os.cpu_count()

print('CPUs being used:',num_cpu)
p = multiprocessing.Pool(processes=num_cpu)
scraped_data = p.map(web_scraper_func.get_data_text, languages)

df=pd.DataFrame(data=scraped_data[0],columns=[languages[0]])
for (data_temp,language) in zip(scraped_data[1:],languages[1:]):
    df_temp=pd.DataFrame(data=data_temp,columns=[language])
    df=pd.concat([df, df_temp], axis=1)

In [20]:
lang_pred = []
website_language=['English','हिंदी','ગુજરાતી','મરાઠી','ಕನ್ನಡ','മലയാളം','தமிழ்','తెలుగు','বাংলা','অসমীয়া','ਪੰਜਾਬੀ','ଓଡ଼ିଆ']
for column in website_language:
    lang_pred.append(detect(' '.join(list(df.loc[:2000,column]))))

lang_name=['English','Hindi','Gujarati','Marathi','Kannada','Malayalam','Tamil','Telugu','Bengali','Assamese','Punjabi','Odia']
df_languages=pd.concat([pd.DataFrame(data=website_language,columns=['Website language']), pd.DataFrame(data=lang_name,columns=['Actual name in English']), pd.DataFrame(data=lang_pred,columns=['Predicted Language'])], axis=1)

print('Language Prediction:')
df_languages.head(22)

Language Prediction:


,Website language,Actual name in English,Predicted Language
0,English,English,en
1,हिंदी,Hindi,hi
2,ગુજરાતી,Gujarati,gu
3,મરાઠી,Marathi,mr
4,ಕನ್ನಡ,Kannada,kn
5,മലയാളം,Malayalam,ml
6,தமிழ்,Tamil,ta
7,తెలుగు,Telugu,te
8,বাংলা,Bengali,bn
9,অসমীয়া,Assamese,bn



Currently, only above 11 Indic languages are supported by IndicTrans -


In [ ]:
df_new=df[['English','हिंदी','ગુજરાતી','મરાઠી','ಕನ್ನಡ','മലയാളം','தமிழ்','తెలుగు','বাংলা','অসমীয়া','ਪੰਜਾਬੀ','ଓଡ଼ିଆ']]
df_new.columns=['English','Hindi','Gujarati','Marathi','Kannada','Malayalam','Tamil','Telugu','Bengali','Assamese','Punjabi','Odia']

In [ ]:
df_new.to_csv(save_dir+'/poshantracker_text_with_href_12_lang.csv',index=False)

In [ ]:
df=pd.read_csv(save_dir+'/poshantracker_text_with_href_12_lang.csv')

A few adjustments are done to accomodate discrepancies for number of rows across languages

In [ ]:
columns=['Hindi','Gujarati','Malayalam','Tamil','Assamese','Punjabi','Odia']
df.loc[137,columns]=df.loc[137,columns]+' '+df.loc[139,columns]
for column in columns:
    df.loc[139:2856,column]=list(df.loc[140:2857,column])
    
columns=['English','Gujarati','Kannada','Malayalam','Tamil','Telugu','Bengali','Assamese','Punjabi','Odia']
for column in columns:
    df.loc[646:2821,column]=list(df.loc[682:2857,column])

columns=['English','Hindi','Gujarati','Marathi','Malayalam','Telugu','Bengali','Assamese','Punjabi']
for column in columns:
    df.loc[662:2856,column]=list(df.loc[663:2857,column])
    
columns=['Hindi','Gujarati','Malayalam','Tamil','Assamese','Punjabi','Odia']
df.loc[1004,columns]=df.loc[1004,columns]+' '+df.loc[1006,columns]
for column in columns:
    df.loc[1006:2856,column]=list(df.loc[1007:2857,column])
    
columns=['English','Hindi','Gujarati','Marathi','Kannada','Malayalam','Tamil','Telugu','Assamese','Punjabi','Odia']
for column in columns:
    df.loc[1032:2856,column]=list(df.loc[1033:2857,column])
    
columns=['English','Hindi','Gujarati','Marathi','Kannada','Malayalam','Tamil','Bengali','Assamese','Punjabi','Odia']
for column in columns:
    df.loc[1067:2856,column]=list(df.loc[1068:2857,column])
    
for column in df.columns:
    df.loc[1073:2856,column]=list(df.loc[1074:2857,column])
    
columns=['English','Gujarati','Marathi','Kannada','Malayalam','Tamil','Telugu','Bengali','Assamese','Punjabi','Odia']
for column in columns:
    df.loc[1079:2856,column]=list(df.loc[1080:2857,column])

In [ ]:
df.to_csv(save_dir+'/poshantracker_text_final.csv',index=False)

In [ ]:
df=pd.read_csv(save_dir+'/poshantracker_text_final.csv')

<h1><center>Task 3 (IndicTrans)</center></h1>

Translation Inference using IndicTrans model: (11 languages currently supported)

In [ ]:
from fairseq import checkpoint_utils, distributed_utils, options, tasks, utils

In [ ]:
os.chdir(path_dir)
from indicTrans.inference.engine import Model

indic2en_model = Model(expdir='../indic-en')

1000 rows are used for inference

In [ ]:
df_final=df[df['English'].astype(str).str[0]!='!']
df_final=df_final.iloc[:1000]
print(df_final.shape)
df_final=df_final.reset_index(drop=True)
lang_codes=['en','hi','gu','mr','kn','ml','ta','te','bn','as','pa','or']

In [ ]:
def print_progress(i,num_batches):
    if i+1<num_batches:
        print(int((i+1)*100/num_batches),'%',end='\r')
    else:
        print(int((i+1)*100/num_batches),'%')
        
def batch_pred(i,out,indic2en_model,df_final,column,language_code,batch_size,num_batches):
    out.extend(indic2en_model.batch_translate(list(df_final.loc[batch_size*i:batch_size*(i+1)-1,column]), language_code , 'en'))
    torch.cuda.empty_cache()
    #print('extended size:',len(out))
    if i%5==0:
        print_progress(i,num_batches)
    return out
    
def single_lang_pred(column,language_code,indic2en_model,df_final,df_out,batch_size,num_batches):
    out=[]
    for i in range(num_batches): 
        out=batch_pred(i,out,indic2en_model,df_final,column,language_code,batch_size,num_batches)
    df_temp=pd.DataFrame(data=out,columns=[column])
    df_out=pd.concat([df_out, df_temp], axis=1)
    return df_out

In [ ]:
num_batches=int(df_final.shape[0]/batch_size)
df_out=pd.DataFrame(data=df_final['English'],columns=['English'])
for (column,language_code) in zip(df_final.columns[1:],lang_codes[1:]):
    print(column,':')
    df_out=single_lang_pred(column,language_code,indic2en_model,df_final,df_out,batch_size,num_batches)
    torch.cuda.empty_cache()

In [ ]:
df_out.to_csv(save_dir+'/poshantracker_text_inference_model1.csv',index=False)

In [7]:
df_out=pd.read_csv(save_dir+'/poshantracker_text_inference_model1.csv')

In [22]:
df_out.loc[95:100,:]

,English,Hindi,Gujarati,Marathi,Kannada,Malayalam,Tamil,Telugu,Bengali,Assamese,Punjabi,Odia
95,Beneficiary count report : Number of records b...,Beneficiary Census Report: Number of records o...,Beneficiary Enumeration Report: Number of reco...,Beneficiary Enumeration Report: Number of Inac...,Beneficiary Enumeration Report: Number of inop...,Number of Beneficiaries reported: Number of re...,User-NUMBER REPORT: Number of dormant entries ...,Beneficiary Enumeration Report: Number of reco...,Number of records inoperative and number of re...,Benefit Count Report: Number of records disabl...,Beneficiary count report: Inactive multiple re...,Beneficiary Census Report: Number of records d...
96,Resources for growth monitoring and general aw...,Resources for Development Monitoring and Gener...,Resources for monitoring growth and general aw...,Resources for Growth Monitoring and General Aw...,Resources for Growth Monitoring and General Aw...,Resources for Growth Monitoring and Public Awa...,Resources for Growth Monitoring and General Un...,Resources for Growth Monitoring and General Aw...,Resources for Growth Monitoring and General Aw...,Resources for development monitoring and gener...,Sources for Development Monitoring and General...,Resources for Growth Monitoring and Public Awa...
97,Growth monitoring charts,Development Supervision Chart,Table relating to Growth Monitoring,Growth Monitoring Chart,Growth Monitoring Chart,Growth Monitoring Charts,Growth Monitoring Maps,Growth Monitoring Maps,Growth Monitoring Chart,Development Monitor Chart,Development Monitoring Chart,Growth Monitoring Charts
98,Poshan Calculator,Nutrition Calculator / Mapping,Nutritional Calculator,Nutritional Calculator,Nutrition Calculator,Position Calculator,POSHAN Calculator,POSHAN calculator,Poshan Calculator,Nutrition Calculator,Nutritional Calculator (Calculator),Page Calculator
99,Poshan Calculator is designed to measure growt...,Nutritional measurement was developed to measu...,The POSHAN calculator is designed to measure t...,The POSHAN calculator is designed to measure t...,The nutrition calculator is designed to measur...,The POSHAN calculator is designed to measure t...,The POSHAN calculator is designed to measure t...,The POSHAN calculator has been developed to ca...,The Poshan calculator is designed to measure t...,The POSHAN Calculator is designed to support t...,The nutrition calculator is designed to measur...,The Parental calculator has been designed to m...
100,How to use Poshan Calculator?,How to use a nutrition meter?,How to use a calculator?,How to use a calculator?,How to use a calculator?,How to use a calculator?,How to use POSHAN calculator?,How to use a calculator?,How to use calculator?,How should the nutrition calculator be used?,How to use a calculator?,How to use a calculator?


<h1><center>Task 4 (HuggingFace)</center></h1>

Translation Inference using HuggingFace model:

In [4]:
from transformers import pipeline

In [6]:
translator = pipeline("translation", model="Helsinki-NLP/opus-mt-mul-en",device=device,batch_size=batch_size)

In [7]:
df=pd.read_csv(save_dir+'/poshantracker_text_final.csv')

1000 rows are used for inference

In [8]:
df_final=df[df['English'].astype(str).str[0]!='!']
df_final=df_final.iloc[:1000]
print(df_final.shape)
df_final=df_final.reset_index(drop=True)

lang_codes=['en','hi','gu','mr','kn','ml','ta','te','bn','as','pa','or']      # although redudent in this case

(1000, 12)


In [9]:
def print_progress(i,num_batches):
    if i+1<num_batches:
        print(int((i+1)*100/num_batches),'%',end='\r')
    else:
        print(int((i+1)*100/num_batches),'%')
        
def batch_pred(i,out,translator,df_final,column,language_code,batch_size,num_batches):
    result=translator(list(df_final.loc[batch_size*i:batch_size*(i+1)-1,column]))
    for dict_text in result:
        out.append(dict_text['translation_text'])
    torch.cuda.empty_cache()
    #print('extended size:',len(out))
    if i%5==0:
        print_progress(i,num_batches)
    return out
    
def single_lang_pred(column,language_code,translator,df_final,df_out,batch_size,num_batches):
    out=[]
    for i in range(num_batches): 
        out=batch_pred(i,out,translator,df_final,column,language_code,batch_size,num_batches)
    df_temp=pd.DataFrame(data=out,columns=[column])
    df_out=pd.concat([df_out, df_temp], axis=1)
    return df_out

In [ ]:
num_batches=int(df_final.shape[0]/batch_size)
df_out=pd.DataFrame(data=df_final['English'],columns=['English'])
for (column,language_code) in zip(df_final.columns[1:],lang_codes[1:]):
    print(column,':')
    df_out=single_lang_pred(column,language_code,translator,df_final,df_out,batch_size,num_batches)
    torch.cuda.empty_cache()

In [12]:
df_out.to_csv(save_dir+'/poshantracker_text_inference_model2.csv',index=False)

In [23]:
df_out=pd.read_csv(save_dir+'/poshantracker_text_inference_model2.csv')

In [24]:
df_out.loc[95:100,:]

,English,Hindi,Gujarati,Marathi,Kannada,Malayalam,Tamil,Telugu,Bengali,Assamese,Punjabi,Odia
95,Beneficiary count report : Number of records b...,Product Count Report: Number of accounts to be...,Labyrinth Accounting Report: The related recor...,Calculates the number of compounding points fo...,Number of results: the number of active logs a...,Report Number of Puzzles: The number of unplay...,User Number Report: The number of accounts tha...,Capitalize report: The number of cards that ar...,Number of license numbers: Number of records d...,Unconfidential Count Report: Number of records...,Useful Count Reports: Achieving a number of re...,Credits to the translators. This string should...
96,Resources for growth monitoring and general aw...,A resource for development monitoring and gene...,Ways to observe the age and the general public,A resource forgroth monitoring and general vig...,Development Supervision and Natural Preservati...,Obstacles to Growth and General Dissemination,Developments to monitor growth and public unde...,Sources for Advanced Monitoring and General Un...,Advanced monitoring and general monitoring tools,A resource for development monitoring and gene...,A Source of Growth and Common Watchfulness,Watching for extra monitors and general warnings
97,Growth monitoring charts,Development monitor chart,The table of contents at the beginning of the ...,Groth monitoring chart,Development Monitoring Charts,Cannot initialise Evolution's mail component.,Development Monitor drawings,Advanced Modes of Watching the World,Up-to-date review chart,Cannot initialise Evolution's mail component.,Cannot initialise Evolution's mail component.,Enhanced monitoring chats
98,Poshan Calculator,Boiling calculator/ scale,Human Calculator,Cannot initialise Evolution's mail component.,Cannot initialise Evolution's mail component.,Poster Calculator,Possian Calculator,Cannot initialise Evolution's mail component.,Cannot initialise Evolution's mail component.,BouncingCalculator,Cannot initialise Evolution's mail component.,Peripherals Calculator
99,Poshan Calculator is designed to measure growt...,The World Health Organization has designed a p...,Human calculator is designed to measure the ch...,The pollutant calculator is designed to measur...,The postage calculator is designed to reduce c...,The World Health Organization (WHO) has been d...,The World Health Organization (WHO) is designe...,The World Health Organization (WHO) has set up...,Based on the values defined by the World Healt...,The World Health Organization (WHO) of the Wor...,The World Health Organization (WHO) has develo...,The peer's calculator is designed to measure t...
100,How to use Poshan Calculator?,How to use the dust measure?,How can you use your personal calculator?,How should we use the washing machine?,How can you use a postal calculator?,How can you use the Poston calculator?,How to use the Poston calculator?,How to use the feed calculator?,How to use the Posan calculator?,How should we use our tongue? How should we us...,How to use a mail calculator?,How to use peer's calculator?


<h1><center>Task 5 (Scoring)</center></h1>

In [14]:
from sacrebleu.metrics import BLEU, CHRF, TER

In [15]:
bleu = BLEU()
chrf = CHRF()

In [22]:
df_out_IndicTrans=pd.read_csv(save_dir+'/poshantracker_text_inference_model1.csv')
df_out_HuggingFace=pd.read_csv(save_dir+'/poshantracker_text_inference_model2.csv')

In [23]:
refs=[list(df_out_HuggingFace['English'])]
scores_bleu_IndicTrans=[]
scores_chrf_IndicTrans=[]
scores_bleu_HuggungFace=[]
scores_chrf_HuggungFace=[]
for column in df_out_HuggingFace.columns[1:]:
    sys1=list(df_out_IndicTrans[column])
    scores_bleu_IndicTrans.append(bleu.corpus_score(sys1, refs).score)
    scores_chrf_IndicTrans.append(chrf.corpus_score(sys1, refs).score)
    sys2=list(df_out_HuggingFace[column])
    scores_bleu_HuggungFace.append(bleu.corpus_score(sys2, refs).score)
    scores_chrf_HuggungFace.append(chrf.corpus_score(sys2, refs).score)

In [24]:
df_temp1=pd.DataFrame(data=df_out_HuggingFace.columns[1:],columns=['Language'])
df_temp2=pd.DataFrame(data=scores_bleu_IndicTrans,columns=['IndicTrans_BLEU'])
df_temp3=pd.DataFrame(data=scores_chrf_IndicTrans,columns=['IndicTrans_CHRF'])
df_temp4=pd.DataFrame(data=scores_bleu_HuggungFace,columns=['HuggingFace_BLEU'])
df_temp5=pd.DataFrame(data=scores_chrf_HuggungFace,columns=['HuggingFace_CHRF'])

In [25]:
df_results=pd.concat([df_temp1, df_temp2,df_temp3,df_temp4,df_temp5], axis=1)

In [27]:
df_results.to_csv(save_dir+'/poshantracker_translation_results_both_models.csv',index=False)

In [26]:
df_results.head(20)

,Language,IndicTrans_BLEU,IndicTrans_CHRF,HuggingFace_BLEU,HuggingFace_CHRF
0,Hindi,37.463862,63.229363,32.188361,51.317558
1,Gujarati,40.203362,63.827258,29.886013,49.510540
2,Marathi,35.249601,63.240492,26.971916,46.786138
3,Kannada,36.995699,60.285479,25.299824,44.039135
4,Malayalam,38.027959,62.791203,22.680365,46.989573
5,Tamil,36.375305,60.167382,25.387703,44.418505
6,Telugu,42.210720,66.698036,31.473089,50.869010
7,Bengali,42.447655,67.414840,27.451132,48.254301
8,Assamese,39.329497,61.974933,29.393651,49.319043
9,Punjabi,37.040062,64.860026,28.643374,49.691875
